<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/main/10_Fold_Cross_Validaiton_for_LSTM_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import time
from sklearn.model_selection import StratifiedKFold
import os
import warnings
warnings.filterwarnings('always')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


# Load Dataset

In [ ]:
df = pd.read_csv("/content/Labeled Posts - preprocessed_csv.csv")
# check class distribution
df['labels'].value_counts(normalize = True)

1.0    0.522571
4.0    0.345714
2.0    0.052286
3.0    0.043429
0.0    0.028286
5.0    0.007714
Name: labels, dtype: float64

In [ ]:
for i in df['selftext']:
  print(i)
  break

Comment and post if you have any questions about the sub.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
X,y = df['selftext'][0:3500].values,df['labels'][0:3500].values
x_train,x_test,y_train,y_test = train_test_split(X,y,stratify=y, test_size=0.1) #set test size to 10% of dataset/train size to 90% --> split train set into train + val data

# Preprocess/Tokenization
(Convert text into numbers)



In [ ]:
def preprocess_string(s):
    # replace digits with no space
    s = re.sub(r"\d", '', s)
    split = re.findall(r"[\w']+|[!?]", s)
    i = 0
    while i < len(split)-1:
      if split[i] == split[i+1]:
        del split[i+1]
        i-=1
      i+= 1
    # Remove all instances of \n where only the n remains
    split = [i for i in split if i != "n"]
    return " ".join(split)

def tokenize(x_train,y_train,x_val,y_val):
    word_list = []
    stop_words = set(stopwords.words('english'))
    for sent in x_train:
        for word in sent.lower().split():
            word = preprocess_string(word)
            if word not in stop_words and word != '':
                word_list.append(word)
    corpus = Counter(word_list)
    # sorting on the basis of most common words
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]                #Only using top 1000 words to train model
    # creating a dict
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_)}
    # tokenize
    final_list_train,final_list_test = [],[]
    for sent in x_train:
            final_list_train.append([onehot_dict[preprocess_string(word)]  for word in sent.lower().split()
                                     if preprocess_string(word) in onehot_dict.keys()])
    for sent in x_val:
            final_list_test.append([onehot_dict[preprocess_string(word)] for word in sent.lower().split()
                                    if preprocess_string(word) in onehot_dict.keys()])
    encoded_train = [1 if label == 2.0 else 0 for label in y_train]            #  Changed from 'positive' to 2.0 for my dataset
    encoded_test = [1 if label == 2.0 else 0 for label in y_val]
    return np.array(final_list_train), np.array(encoded_train),np.array(final_list_test), np.array(encoded_test),onehot_dict

In [ ]:
x_train,y_train,x_test,y_test,vocab = tokenize(x_train,y_train,x_test,y_test)

<ipython-input-6-13fe6daea044>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(final_list_train), np.array(encoded_train),np.array(final_list_test), np.array(encoded_test),onehot_dict


# Padding (len 500)


In [ ]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features
x_train_pad = padding_(x_train,500)
x_test_pad = padding_(x_test,500)

# Shuffle Train/Val_data

> Get 10 folds (split train_data into validation & train set - 1:9 ratio)



In [ ]:
batch_size = 35    #  Number of posts per batch (when training)
def shuffler(x, y):
  data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y))
  # make sure to SHUFFLE your data
  #train_loader is a list of len 63 - we divided 2835 posts in x_train into 81 batchs of 35 posts
  #each of the 63 lists are made of 2 sublists, both of len 35 - first is the padded texts, second is the list of labels
  loader = DataLoader(data, shuffle=True, batch_size=batch_size)           # shuffled train_data/test_data
  return loader

In [ ]:
skf = StratifiedKFold(n_splits=10)

train_loader, valid_loader = [], []
for i, (train_index, test_index) in enumerate(skf.split(x_train_pad, y_train)):
  #x_train, y_train, x_val, y_val
  a, b, c, d = [], [], [], []
  for j in train_index:
    a.append(x_train_pad[j])
    b.append(y_train[j])
  for j in test_index:
    c.append(x_train_pad[j])
    d.append(y_train[j])
  a, b, c, d = map(np.array, [a, b, c, d])
  train = shuffler(a, b)
  val = shuffler(c, d)
  train_loader.append(train)
  valid_loader.append(val)
print(len(train_loader), len(valid_loader), len(train_loader[0]), len(valid_loader[0]))

test_loader = shuffler(x_test_pad, y_test)

#train_loader: 10 folds with 81 batches of 35 posts, valid_loader: 10 folds with 9 batches of 35 posts

#Train: 2835 posts, Validation: 315 posts, Test: 350 posts

10 10 81 9
350


# Create Neural Network Model

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, no_layers, vocab_size, hidden_dim, embedding_dim, drop_prob=0.5):
        super(SentimentRNN,self).__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers
        self.vocab_size = vocab_size
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim, num_layers=no_layers, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(0.3)
        # linear and sigmoid layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
    def forward(self,x,hidden):
        batch_size = x.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True
        #print(embeds.shape)  #[50, 500, 1000]
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        # return last sigmoid output and hidden state
        return sig_out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden
no_layers = 2
vocab_size = len(vocab) + 1 #extra 1 for padding
embedding_dim = 64
output_dim = 1
hidden_dim = 256
model = SentimentRNN(no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5)

# Define learning rate and accuracy func

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def acc(pred,label):
  pred = torch.round(pred.squeeze())
  pred = pred.to("cpu").detach().numpy().squeeze()
  label = label.to("cpu").detach().numpy().squeeze()
  return accuracy_score(label, pred)

def f1(pred, label):
  pred = torch.round(pred.squeeze())
  pred = pred.to("cpu").detach().numpy().squeeze()
  label = label.to("cpu").detach().numpy().squeeze()
  return f1_score(label, pred, labels=np.unique(pred))

# Train Model

In [ ]:
def stop(acc_list):
  a = 0
  for i in range(len(test_acc), len(test_acc)-10, -1):
    if acc_list[i] > acc_list[i-1]: #check if loss is INCREASING (getting worse)
      a += 1
  return True if a > 7 else False

In [ ]:
def test(model, test_loader):
  test_h = model.init_hidden(batch_size)
  test_losses = []
  test_acc = 0.0
  model.eval()
  c = 0
  t5 = 0
  for ind, (inputs, labels) in enumerate(test_loader):
    test_h = tuple([each.data for each in test_h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, test_h = model(inputs, test_h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    accuracy = f1(output,labels)
    #add f1 for test predictions
    test_acc += accuracy
    # #check if loss is increasing every 10 posts; save model every 5 posts
    # if ind % 5 == 0 and i != 0:
    #   save = True
    #   c += 1
    #   if c == 2:
    #     if t5 > test_loss.item():
    #       save = False
    #     if stop(test_acc):
    #       break
    #     c = 0
    #   if save:
    #     torch.save(model.state_dict(), f'/content/saved_model{ind}.pt')
    #     if ind != 5: # if ind != 5 AND save == True (meaning if we are saving a new model after 5 iterations, delete the previously saved model)
    #       os.remove(f'/content/saved_model{ind-5}.pt')
    #   t5 = test_loss.item()

  return test_losses, test_acc/len(test_loader)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def train(model, train_loader, valid_loader):
  clip = 5
  epochs = 100
  valid_loss_min = np.Inf
  # train for some number of epochs
  epoch_tr_loss,epoch_vl_loss = [],[]
  epoch_tr_acc,epoch_vl_acc = [],[]
  time = 0
  for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0
    model.train()
    # initialize hidden state
    h = model.init_hidden(batch_size)
    for ind, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        model.zero_grad()
        output,h = model(inputs,h)
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        train_losses.append(loss.item())
        # calculating accuracy
        accuracy = acc(output,labels)
        train_acc += accuracy
        # clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
    #Validation
    val_losses, val_acc = test(model, valid_loader)

    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_loader)
    epoch_val_acc = val_acc    #don't divide by len(valid_loader) because it's already done in the test() method

    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
  #return train (acc/loss), val (acc/loss)
  return max(epoch_tr_acc), min(epoch_tr_loss), max(epoch_vl_acc), min(epoch_vl_loss)   # each list has 100 values (1 per epoch): each value is the average acc/loss after training on entire dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(train_loader[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
total_t = 0
best_model = model
best_val_loss = 10000000000
#Loss/accuracy results for
train_losses, train_accs, val_losses, val_accs, test_losses, test_acc = [], [], [], [], [], []
for i in range(10):
  a = time.time()
  #moving to gpu
  model.to(device)
  #Train
  epoch_tr_acc, epoch_tr_loss, epoch_vl_acc, epoch_vl_loss = train(model, train_loader[i], valid_loader[i])
  train_losses.append(float(f'{epoch_tr_loss:.3f}'))
  train_accs.append(float(f'{epoch_tr_acc:.3f}'))
  val_losses.append(float(f'{epoch_vl_loss:.3f}'))
  val_accs.append(float(f'{epoch_vl_acc:.3f}'))
  #you won't need test_losses - this is only used for validations step
  test_losses, test_accuracy = test(model, test_loader)
  test_acc.append(float(f'{test_accuracy:.3f}'))
  if epoch_vl_loss < best_val_loss:
    best_model = model
    best_val_loss = epoch_vl_loss
  b = time.time()
  total_t += b-a
  print(f"Fold {i} done")
  model = SentimentRNN(no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5)

torch.save(best_model.state_dict(), '/content/saved_best_model.pt')
print(f"time: {total_t:.3f} seconds")
# below are lists of best accuracies/losses for each fold; AKA max accuracy out of 100 epochs
#Train results
print("Train accuracies:", train_accs)
print("Train losses:", train_losses)
#Val results
print("Val accuracies:", val_accs)
print("Val losses:", val_losses)
print("Best Val loss:", best_val_loss)
#Test results
print("Test accuracies:", test_acc)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fold 0 done


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics

Fold 1 done


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics

...

Fold 3 done


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics

Fold 4 done


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics

Fold 5 done
Fold 6 done
Fold 7 done
Fold 8 done
Fold 9 done
time: 4595.241 seconds
Train accuracies: [0.1590828924162256, 0.9231040564373889, 0.9453262786596113, 0.9195767195767186, 0.9446208112874771, 0.8726631393298051, 0.40670194003527277, 0.06666666666666658, 0.2892416225749556, 0.05396825396825393]
Train losses: [0.7035349592750455, 0.6809246753468926, 0.6744887225421858, 0.6804166679029111, 0.6743434380601954, 0.6841176836578934, 0.6951826067618382, 0.7095220272923694, 0.6977875894970365, 0.7160423008012183]
Val accuracies: [0.10035273368606704, 0.0, 0.0, 0.1111111111111111, 0.0, 0.1111111111111111, 0.11271481258858539, 0.10165428586481219, 0.10368501587666673, 0.10196161073354056]
Val losses: [0.7034615808063083, 0.6815830071767172, 0.6741755670971341, 0.6805333561367459, 0.6742479006449381, 0.6844501031769646, 0.6955255601141188, 0.7095340689023336, 0.698320104016198, 0.7159809271494547]
Best Val loss: 0.6741755670971341
Test accuracies: [0.08698570751202331, 0.0, 0.0, 0.0, 0.0

In [ ]:
def predict_text(texts):
  l = []
  for text in texts:
    word_seq = np.array([vocab[preprocess_string(word)] for word in text.split()
                      if preprocess_string(word) in vocab.keys()])
    word_seq = np.expand_dims(word_seq,axis=0)
    pad =  torch.from_numpy(padding_(word_seq,500))
    inputs = pad.to(device)
    batch_size = 1
    h = model.init_hidden(batch_size)
    h = tuple([each.data for each in h])
    output, h = model(inputs, h)
    l.append(output.item())
  return l

index = 4274
print(df['selftext'][4271])
print('='*70)
print(f'Actual sentiment is  : {df["labels"][4270:4272]}')
print('='*70)
pro = predict_text(df['selftext'][4270:index])
print(pro)
# status = "positive" if pro > 0.5 else "negative"
# pro = (1 - pro) if status == "negative" else pro
# print(f'Predicted sentiment is {status} with a probability of {pro}')

I’ve been meaning to make a post on here for a while, but I’m happy to finally say I’ve been vape free for 61 days now! I’d only been previously vaping for about a year so my case isn’t as bad as some, but I hated how attached I was to my vape and wanted to quit. I knew it wasn’t good for my health and since I’m into some singing and vocal performance, I knew I needed to stop soon. 

Family and friends were encouraging me for a few months but it didn’t help out too much. Sometimes I’d try to stop for a day or two, but I’d see my vape and get right back to it. But, one day I thought “I’m not gonna let this control me.” Instead of “I think I’m going to quit”, I said “I WILL quit vaping.” 

The first couple days it was terrible. By day 3 I got so close to going to buy one, but I thought back to myself and kept going. I will say though, once I made it to 1 week, it got a lot easier from there. I still get headaches and urges even now, but they normally go away after a few minutes. 

For an